In this lesson, we'll pick up where we left off and write a query to select stars from a particular region of the sky.

In [32]:
# Imports
import sys
import astropy.units as u
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from gala.coordinates import GD1Koposov10


Outline:
We'll start with an example that does a "cone search"; that is, it selects stars that appear in a circular region of the sky.

Then, to select stars in the vicinity of GD-1, we'll:

Use Quantity objects to represent measurements with units.

Use Astropy to convert coordinates from one frame to another.

Use the ADQL keywords POLYGON, CONTAINS, and POINT to select stars that fall within a polygonal region.

Submit a query and download the results.

Store the results in a FITS file.

Working with units:

Astropy provides tools for including units explicitly in computations, which makes it possible to detect errors before they cause disasters.

In [5]:
angle = 10 * u.degree
type(angle)
angle

<Quantity 10. deg>

In [6]:
# Convert degrees to arcminutes
angle_arcmin = angle.to(u.arcmin)
angle_arcmin

<Quantity 600. arcmin>

In [9]:
# If you add quantities, Astropy converts them to compatible units if possible
angle + 30*u.arcmin

<Quantity 10.5 deg>

In [11]:
# If units are not compatible
angle + 5*u.second # UnitConversionError

Excercise:

Create a quantity that represents 5 arcminutes and assign it to a variable called radius.

Then convert it to degrees.

In [20]:
radius = 5*u.arcmin
radius

<Quantity 5. arcmin>

In [22]:
radius.to(u.degree)

<Quantity 0.08333333 deg>

Selecting a region:

One of the most common ways to restrict a query is to select stars in a particular region of the sky

Here's a query from the Gaia archive documentation that selects objects in a circular region centered at (88.8, 7.4) with a search radius of 5 arcmin (0.08333 deg).

In [27]:
query_cone = """
SELECT TOP 10
source_id
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
    POINT(ra,dec),
    CIRCLE(88.8, 7.4, 0.083333))
"""

# POINT, CIRCLE, and CONTAINS are specific to ADQL

POINT: a location in ICRS coordinates, specified in degrees of right ascension and declination.

CIRCLE: a circle where the first two values are the coordinates of the center and the third is the radius in degrees.

CONTAINS: a function that returns 1 if a POINT is contained in a shape and 0 otherwise.

In [28]:
job = Gaia.launch_job(query_cone)
job

In [30]:
results = job.get_results()
results

source_id
int64
3322774652250782080
3322774686612065024
3322774617891039744
3322774480451102080
3322774068134269184
3322774446092909184
3322774102496541440
3322773892042964224
3322774446092380800


Excercise:

You can use TOP to limit the size of the results, but then you still don't know how big the results will be

An alternative is to use COUNT, which asks for the number of rows that would be selected, but it does not return them.

In the previous query, replace TOP 10 source_id with COUNT(source_id) and run the query again. How many stars has Gaia identified in the cone we searched?

In [31]:
query_cone = """
SELECT 
COUNT(source_id)
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
    POINT(ra,dec),
    CIRCLE(88.8, 7.4, 0.083333))
"""

job = Gaia.launch_job(query_cone)
results = job.get_results()
results

COUNT
int64
594


Answer: 594 stars


Getting GD-1 Data

# Attempting to reproduce figure 1 from the paper

Transforming coordinates:

Astropy provides a SkyCoord object that represents sky coordinatyes relative to a specified frame

In [35]:
# Creating a SkyCoord object that represents
# the apporx coordinates of Betelgeuse in the ICRS frame

ra = 88.8 * u.degree
dec = 7.4 * u.degree

coord_icrs = SkyCoord(ra=ra, dec=dec, frame="icrs")
coord_icrs

<SkyCoord (ICRS): (ra, dec) in deg
    (88.8, 7.4)>

We can transform coord_icrs to galactic coordinates:

In [37]:
coord_galactic = coord_icrs.transform_to("galactic")
coord_galactic

<SkyCoord (Galactic): (l, b) in deg
    (199.79693102, -8.95591653)>

To transform to and from GD-1 coordinates, we'll use a framed defined by Gala (an Astropy affiliated library that provides t 